In [ ]:
import glob
import pdb
import cProfile
import warnings
import networkx as nx
from lxml import etree
from graphs.call_graph.c import build_call_graph
from graphs.devrank import devrank
from graphs.call_commit_graph import CallCommitGraph
warnings.simplefilter('ignore')

In [ ]:
c_roots = []
for xml in glob.glob('../repos/linux-kernel-xml/kernel/*.c.xml'):
    tree = etree.parse(xml)
    c_roots.append(tree.getroot())
G, _, func_to_file = build_call_graph(c_roots)
print("Number of nodes: {}".format(len(G.nodes())))
print("Number of edges: {}".format(len(G.edges())))
print("Number of connected components: {}".format(nx.number_weakly_connected_components(G)))

In [ ]:
pr = devrank(G, alpha=0.85)
sorted_pr = sorted(pr.items(), key=lambda x: x[1])
# draw_call_graph(G)
[(x[0], x[1], func_to_file[x[0]] if x[0] in func_to_file else 'Unknown') for x in reversed(sorted_pr[-20:])]

In [ ]:
pr = devrank(G, alpha=0.1)
sorted_pr = sorted(pr.items(), key=lambda x: x[1])
# draw_call_graph(G)
[(x[0], x[1], func_to_file[x[0]] if x[0] in func_to_file else 'Unknown') for x in reversed(sorted_pr[-20:])]

In [ ]:
g = CallCommitGraph('../repos/linux-kernel')
cProfile.run("g.process(rev='v2.6.12', num_commits=500)")
g.devrank_functions(0.85)

In [ ]:
def checker(analyzer):
    for func_name in analyzer.G:
        edits_sum = 0
        seen = False
        for sha in analyzer.history:
            if func_name in analyzer.history[sha]:
                seen = True
                edits_sum += analyzer.history[sha][func_name]
        if seen:
            try:
                assert(edits_sum == analyzer.G.node[func_name]['num_lines'])
            except:
                print(func_name)
                print('edit_sum: %d'%(edits_sum,))
                print('num_lines: %d'%(analyzer.G.node[func_name]['num_lines'],))
                pdb.set_trace()
                
def order_checker(analyzer, every_n):
    devrank = analyzer.devrank()
    locrank = analyzer.locrank()
    assert(len(devrank) == len(locrank))
    counter = 0
    for i in range(len(devrank)):
        if devrank[i][0] != locrank[i][0]:
            counter += 1
            
    left = len(devrank)
    ptr = 0
    diff = []
    while left >= every_n:
        set1 = set([x[0] for x in devrank[ptr:ptr+every_n]])
        set2 = set([y[0] for y in locrank[ptr:ptr+every_n]])
        diff.append(every_n - len(set1 & set2))
        ptr += every_n
        left -= every_n
        
    change = []
    for i in range(len(locrank)):
        sha = locrank[i][0]
        for j in range(len(devrank)):
            if sha == devrank[j][0] and i != j:
                change.append([sha, locrank[i][1], i, j])
    return counter, diff, change

In [ ]:
checker(analyzer)

In [ ]:
order_checker(analyzer, 3)